In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import string

import nltk
from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
path = '/home/manikya_varshney/Documents/Python/Yale/final_processed_h01-20200912-101538.csv'
data = pd.read_csv(path, low_memory=False)

In [3]:
data

,id,user_id,extended_tweet_full_text,extended_tweet_full_text_duplicate,final
0,1.304786e+18,9.053900e+07,woke up to see if the justin bieber pandemic w...,woke up to see if the justin bieber pandemic w...,woke see justin bieber pandemic go back sleep
1,1.304786e+18,8.497239e+17,“our intention is to make sure that evidence s...,@TeresaCCarter2 “Our intention is to make sure...,“ intention make sure evidence sciencebased da...
2,1.304786e+18,1.293830e+18,for more information contact us mail follow ...,For More Information contact us. \nMail:- digi...,information contact u mail follow 📷instagram g...
3,1.304786e+18,1.188902e+18,uae reports 1007 new covid19 cases highest sin...,"UAE reports 1,007 new Covid-19 cases, highest ...",uae report 1007 new covid19 case high since ou...
4,1.304786e+18,2.273830e+08,trump officials interfered with cdc reports on...,Trump officials interfered with CDC reports on...,trump official interfere cdc report covid19 po...
...,...,...,...,...,...
6732,1.304427e+18,6.874206e+07,why did twitter suddenly reinstate could it ...,Why did Twitter suddenly reinstate @clif_high?...,twitter suddenly reinstate could science valid...
6733,1.304671e+18,8.323244e+17,denna veckas covid19 veckorapport från folkhäl...,Denna veckas COVID-19 veckorapport från Folkhä...,denna veckas covid19 veckorapport från folkhäl...
6734,1.304768e+18,4.446656e+09,republicans defend trump after he admitted dow...,Republicans Defend Trump After He Admitted Dow...,republican defend trump admit downplay true th...
6735,1.301853e+18,3.914277e+08,the recession on the back of the governments h...,The recession on the back of the Government's ...,recession back government handle covid economi...


In [4]:
keywords = ['stay at home' , 'do your part', 'Responsible',
            'home', 'house', 'cancel', 'shutdown', 'postpone',
            'school closure', 'Closure', 'business closure',
            'suspension', 'quarantine', 'lockdown', 'social distance', 
            'social distancing', 'self quarantine', 'isolat', '6-feet',
            'distance', '#clubquarantine', '#quarantinelife', '#quarantineacitivites']

##### 1. Casing (Upper or lower case)
##### 2. Noise Removal (Removal of punctuation, white spaces, special characters, HTML tags)
##### 3. Tokenization (Tweets to tokens i.e. words seprated by spaces)
##### 4. Stopword Removal
##### 5. Text Normalization (Stemming and Lemmatization)

In [5]:
#Convert to lower
for i in range(len(keywords)): 
    keywords[i] = keywords[i].lower()

#Remove punctuations   
for i in range(len(keywords)): 
    keywords[i] = keywords[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords)): 
    keywords[i] = keywords[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_tokens = [sub.split() for sub in keywords] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_filtered=remove_stopwords(keywords)

#Stemming
ps = PorterStemmer()
keywords_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_filtered]
keywords_stem = [" ".join(sentence) for sentence in keywords_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_filtered]
keywords_lem = [" ".join(sentence) for sentence in keywords_lem]

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
temp=' '.join(data['keywords_lem'].tolist())
wordcloud = WordCloud(width = 800, height = 500, background_color ='white', min_font_size = 10).generate(temp)
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0) 
plt.show()